#### csv files: daily-sales, monitors, stocks-all

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}','pct':'{:,.2f}%',
               'fm_price':'{:,.2f}','to_price':'{:,.2f}',
               'max_price':'{:,.2f}','min_price':'{:,.2f}'}

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2021, 12, 29)

### Yesterday = Last closed work day

In [2]:
#yesterday = today - timedelta(days=3)
yesterday

datetime.date(2021, 12, 29)

### Restart and Run All Cells

In [3]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
round((to_price-fm_price)*1.0/fm_price*100.0,2) AS pct,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a JOIN stocks t ON a.name = t.name 
WHERE to_date >= "%s" AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
round((to_price-fm_price)*1.0/fm_price*100.0,2) AS pct,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a JOIN stocks t ON a.name = t.name 
WHERE to_date >= "2021-12-29" AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [4]:
df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.shape

(25, 10)

### Price change up or down more than 3%

In [5]:
#df['bdays'] = np.busday_count(df['fm_date'].dt.date,df['to_date'].dt.date)
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.pct) >= 3,
    ['pct','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="pct",ascending=False)

,pct,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
STA,5.04,B,29.75,31.25,SET50,2021-12-20,9,31.75,29.75
SYNEX,4.44,O,33.75,35.25,SET100,2021-12-28,1,35.25,33.75
TOP,3.11,I,48.25,49.75,SET50,2021-12-27,2,49.75,48.50


In [6]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/daily-watch-list.csv',
 '\\Users\\User\\iCloudDrive\\daily-watch-list.csv',
 '\\Users\\User\\Dropbox\\daily-watch-list.csv')

In [7]:
watch_list.sort_values(['status','pct'],ascending=[True,True]).to_csv(output_file,header=None)
watch_list.sort_values(['status','pct'],ascending=[True,True]).to_csv(data_file,header=None)
watch_list.sort_values(['status','pct'],ascending=[True,True]).to_csv(box_file)

### Create daily-sales from sales

In [8]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df.sort_values(['status','pct'],ascending=[True,True]).to_csv(output_file,header=None)
df.sort_values(['status','pct'],ascending=[True,True]).to_csv(data_file,header=None)
df.sort_values(['status','pct'],ascending=[True,True]).to_csv(box_file)

In [9]:
df.shape[0]

25

### Create monitors from stocks

In [10]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.shape

(30, 1)

In [11]:
sql = """
SELECT status, COUNT(*) 
FROM stocks 
WHERE status IN ("B","I","O","S")
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,COUNT(*)
0,B,8
1,I,15
2,O,6
3,S,1


### output_file = iCloud Drive on PC

In [12]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)

### End of daily process

In [13]:
sql = """
SELECT status, COUNT(*) 
FROM stocks 
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,COUNT(*)
0,B,8
1,I,15
2,O,6
3,S,1
4,T,4
5,U,6
6,X,13


In [14]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
DOHOME,B,SET100
GLOBAL,B,SET50
NOBLE,B,SET
PTT,B,SET50
RATCH,B,SET50
SIS,B,SET
STA,B,SET50
TISCO,B,SET50
BGRIM,I,SET50


In [15]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [16]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BGC,O
DOHOME,B
GLOBAL,B
NOBLE,B
PTT,B
RATCH,B
ROJNA,O
SIS,B
STA,B


In [17]:
buy_candidates.shape[0]

14

In [18]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BGRIM,I
CKP,I
DCC,I
DIF,I
IMH,I
IVL,I
KBANK,I
NER,I
PTTGC,S


In [19]:
sell_candidates.shape[0]

16